In [1]:
# Google Colab-only setup. No need to run this cell in other environments.
use_colab = False

if use_colab:
    # Mount my Google Drive root folder
    from google.colab import drive
    drive.mount('/content/drive')

    # cd to bayesian-dl-experiments directory
    %cd 'drive/My Drive/Colab Notebooks/bayesian-dl-experiments'
    !ls

## Experiment Setup

In [2]:
import torch
import numpy as np

# IPython reloading magic
%load_ext autoreload
%autoreload 2

# Random seeds
# Based on https://pytorch.org/docs/stable/notes/randomness.html
torch.manual_seed(682)
np.random.seed(682)

# torch.device / CUDA Setup
use_cuda = True

if use_cuda and torch.cuda.is_available():
    torch_device = torch.device('cuda')
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
else:
    torch_device = torch.device('cpu')

# Dataset to use
dataset_name = 'yacht'

# Training set size
dataset_train_size = 0.8

# L2 regularization strength
reg_strength = 0.01

# Epochs
n_epochs = 10

# Data batch sizes
n_training_batch = 10
n_test_batch = 10

# Number of test predictions (for each data point)
n_predictions = 10000

## Prepare data

### Get the data as a torch Dataset object

In [3]:
from ronald_bdl import datasets

if dataset_name == 'MNIST':
    dataset = datasets.MNIST(root_dir='./datasets_files', transform=None, download=True)
else:
    dataset = datasets.UCIDatasets(dataset_name, root_dir='./datasets_files', transform=None, download=True)

# Print the size of the dataset
print("dataset size = " + str(dataset.data.shape))

Using downloaded and verified file: ./datasets_files/yacht/yacht_hydrodynamics.data
dataset size = torch.Size([308, 7])


### Split into training/test set using torch's DataLoader

In [4]:
from torch.utils.data import random_split, DataLoader

train_size = int(dataset_train_size * len(dataset))
test_size = len(dataset) - train_size

train, test = random_split(dataset, lengths=[train_size, test_size])

train_loader = DataLoader(train, batch_size=n_training_batch)
test_loader = DataLoader(test, batch_size=n_test_batch)

## Define network

In [5]:
from ronald_bdl import models

network = models.FCNetMCDropout(
    input_dim=len(dataset.features), 
    output_dim=len(dataset.targets),
    hidden_dim=100,
    n_hidden=2,
    dropout_rate=0.01,
)

# Send the whole model to the selected torch.device
network.to(torch_device)

# Print the network structure
print(network)

FCNetMCDropout(
  (input): Linear(in_features=6, out_features=100, bias=True)
  (hidden_layers): ModuleList(
    (0): Linear(in_features=100, out_features=100, bias=True)
    (1): Linear(in_features=100, out_features=100, bias=True)
  )
  (output): Linear(in_features=100, out_features=1, bias=True)
)


## Train the network

### Setup optimizer

In [6]:
from torch import nn, optim

# Mean Squared Error for loss function to minimize
objective = nn.MSELoss()

# Adam optimizer
# https://pytorch.org/docs/stable/optim.html?highlight=adam#torch.optim.Adam
# NOTE: Need to set L2 regularization from here
optimizer = optim.Adam(
    network.parameters(),
    lr=0.01, 
    weight_decay=reg_strength # L2 regularization
)

### Run the optimizer

In [7]:
# Partially adapted from https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
for epoch in range(n_epochs): # loop over the dataset multiple times

    print("Running epoch " + str(epoch))

    for i, data in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, targets = data

        # Store the batch to torch_device's memory
        inputs = inputs.to(torch_device)
        targets = targets.to(torch_device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = network(inputs)

        loss = objective(outputs, targets)
        loss.backward()

        optimizer.step()

Running epoch 0
Running epoch 1
Running epoch 2
Running epoch 3
Running epoch 4
Running epoch 5
Running epoch 6
Running epoch 7
Running epoch 8
Running epoch 9


## Make predictions

In [8]:
for i, data in enumerate(test_loader):
    # Get the test data
    inputs, targets = data

    # Store the batch to torch_device's memory
    inputs = inputs.to(torch_device)
    targets = targets.to(torch_device)
    
    predictions, mean, var, metrics = network.mc_predict(inputs, n_predictions,
                                                         y_test=targets, reg_strength=reg_strength)

    print("Test " + str(i))
    print("Mean = " + str(mean))
    print("Variance = " + str(var))
    
    # Print additional metrics
    if len(metrics) > 0:
        for key, value in metrics.items():
            print(str(key) + " = " + str(value))
    print()

Test 0
Mean = tensor([[ 6.2689],
        [ 3.2416],
        [ 2.2282],
        [10.2802],
        [ 1.9485],
        [ 0.6049],
        [ 8.8386],
        [17.6784],
        [ 3.8787],
        [14.1831]])
Variance = tensor([[0.5864],
        [0.3153],
        [0.1435],
        [0.9196],
        [0.0756],
        [0.0020],
        [0.6851],
        [1.4885],
        [0.3241],
        [1.0319]])
rmse = tensor(11.0499)
test_ll = tensor(-3.8261)

Test 1
Mean = tensor([[ 1.0782],
        [11.1617],
        [ 1.4636],
        [ 1.6507],
        [ 0.6837],
        [ 0.5707],
        [15.4054],
        [10.1735],
        [ 2.3291],
        [10.9230]])
Variance = tensor([[1.6047e-02],
        [9.6459e-01],
        [4.8819e-02],
        [6.2518e-02],
        [3.8455e-03],
        [1.0750e-04],
        [1.0293e+00],
        [1.0077e+00],
        [2.2047e-01],
        [1.2141e+00]])
rmse = tensor(3.7845)
test_ll = tensor(-3.2946)

Test 2
Mean = tensor([[ 3.2637],
        [ 7.0125],
        [20.680